# open CV

TODO: create security cameras

TODO: create a Google VR app that allows you to type on any surface using the front camera






detect people?


In [ ]:
import os
import requests
import cv2
import numpy as np
import_notebook('motion detection', globals())


# Initialize HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def get_image(image_path):

  # Load image
  if os.path.exists(image_path):
    image = cv2.imread(image_path)
  else:
    # Fetch the image from the URL
    response = requests.get(image_path)
    response.raise_for_status()  # Raise an error if the request fails

    # Convert to a NumPy array for OpenCV
    image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

  # Resize image (optional, improves detection speed)
  image = cv2.resize(image, (800, 600))

  return image

def trace_boxes(image_path):

  if isinstance(image_path, str):
    image = get_image(image_path)
  else:
    image = image_path

  # Convert to grayscale (not necessary, but can improve performance)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Detect people
  boxes, weights = hog.detectMultiScale(gray, winStride=(4, 4), padding=(8, 8), scale=1.05)

  return boxes

def trace_people(image_path):

  if isinstance(image_path, str):
    image = get_image(image_path)
  else:
    image = image_path

  boxes = trace_boxes(image)

  # Draw bounding boxes
  for (x, y, w, h) in boxes:
      cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

  return image

def main(image_path):
  prev_frame = None
  image = None
  while True:
    # Create a random image (or load a new one dynamically)
    image = get_image(image_path)

    if(prev_frame is not None):
      frame = image_grayscale(image)
      diff_image = diff_images(prev_frame, frame)
      motion = percent_motion(diff_image)
      # TODO: if motion is > 1.0 start recording and compile a video and upload to youtube
      print(motion)
    else:
      frame = image_grayscale(image)

    image = trace_people(image)  # Random image example

    prev_frame = frame.copy()

    # Display the image
    cv2.imshow("People Detection", image)

    # Wait for 1 second (1000 ms) and check for key press
    if cv2.waitKey(1000) & 0xFF == ord('q'):  # Press 'q' to exit
        break

  cv2.destroyAllWindows()

__all__ = {
  "main": main,
  "trace_people": trace_people
}



motion detection?


In [ ]:

import cv2
import numpy as np

def percent_motion(thresh):
  white_pixels = np.count_nonzero(thresh == 255)
  total_pixels = thresh.size

  # Calculate percentage
  white_percentage = (white_pixels / total_pixels) * 100
  return white_percentage

def image_grayscale(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  return gray_image

def diff_images(prev_frame, frame):
  # Compute absolute difference between current and previous frame
  diff = cv2.absdiff(prev_frame, frame)

  # Threshold the difference image
  _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
  # Update previous frame
  # prev_frame = frame.copy()
  return thresh

__all__ = {
  "percent_motion": percent_motion,
  "image_grayscale": image_grayscale,
  "diff_images": diff_images
}



python write video?


In [ ]:
import cv2
import glob

# Set output video properties
fps = 30
output_file = "output.mp4"

def write_video():
  # Get list of images
  image_files = sorted(glob.glob("frame_*.jpg"))  # Adjust pattern as needed

  # Read first image to get dimensions
  frame = cv2.imread(image_files[0])
  height, width, _ = frame.shape

  # Define VideoWriter object
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec
  out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

  # Write images to video
  for img_path in image_files:
      frame = cv2.imread(img_path)
      out.write(frame)

  # Cleanup
  out.release()
  cv2.destroyAllWindows()

__all__ = {
  "write_video": write_video
}



python write images to ffmpeg stream?

ffmpeg -i http://192.168.4.22:8080/video -i http://192.168.4.22:8080/audio.wav -c:v libx264 -preset ultrafast -b:v 4500k -maxrate 4500k -bufsize 9000k -g 50 -c:a aac -b:a 128k -ar 44100 -f flv rtmp://a.rtmp.youtube.com/live2/dvqv-7eyf-72d5-vaw7-dvsb



In [ ]:
import os
import cv2
import subprocess
import time
import signal
import pyautogui
import numpy as np

# Take a screenshot and save it to a file
screenshot = pyautogui.screenshot()

width, height = screenshot.size

# Video properties
#width, height, fps = 1280, 720, 30
#fps = 4

# FFmpeg command for encoding & saving video
ffmpeg_cmd = [
    "ffmpeg",
    "-y",
    "-f", "image2pipe",
    "-vcodec", "mjpeg",
    "-r", "30",
    "-i", "-",  # Read from stdin
    "-re", # https://stackoverflow.com/questions/64745099/youtube-isnt-receiving-any-data-from-my-ffmpeg-stream-to-the-rtmp-server
    "-f", "lavfi",
    "-i", "anullsrc",
    "-c:v", "libx264",
    "-pix_fmt", "yuv420p",
    "-preset", "ultrafast",
    "-b:v", "4500k",
    "-maxrate", "4500k",
    "-bufsize", "9000k",
    "-g", "50",
    "-c:a", "aac", 
    "-b:a", "128k", 
    "-ar", "44100",
    "-f", "flv",
    # "-fifo_format", "flv",
    
]

def stream_images(streamName):

  args = ffmpeg_cmd + ["rtmp://a.rtmp.youtube.com/live2/" + streamName]
  # Start FFmpeg process
  process = subprocess.Popen(args, stdin=subprocess.PIPE)

  # OpenCV VideoWriter (encoding to a memory buffer)
  # fourcc = cv2.VideoWriter_fourcc(*"H264")
  # video_writer = cv2.VideoWriter(process.stdin, fourcc, fps, (width, height))

  # OpenCV video capture
  #cap = cv2.VideoCapture(0)

  try:
    while True:
      #ret, frame = cap.read()
      #if not ret:
      #    break
      screenshot = pyautogui.screenshot()
      screenshot_np = np.array(screenshot)

      # Convert RGB to BGR (OpenCV uses BGR format)
      screenshot_cv = cv2.cvtColor(screenshot_np, cv2.COLOR_RGB2BGR)

      image = cv2.resize(screenshot_cv, (1920, 1080))

      _, jpeg_bytes = cv2.imencode(".jpg", image, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

      # Write frame to OpenCV VideoWriter
      # video_writer.write(screenshot_cv)

      # Get encoded frame and send to FFmpeg
      for i in range(15):
        process.stdin.write(jpeg_bytes.tobytes())
        time.sleep(1 / 30)


  except KeyboardInterrupt:
    print("\nCTRL+C detected. Exiting...")
  # Cleanup
  # cap.release()
  # video_writer.release()
  process.stdin.close()
  #os.killpg(process.pid, signal.SIGTERM) 
  os.killpg(os.getpgid(process.pid), signal.SIGTERM) 
  process.wait()

__all__ = {
  "stream_images": stream_images
}



create a live streaming request?


In [ ]:
import os
from googleapiclient.discovery import build
from google.oauth2 import service_account

HOME_DIR = os.environ.get("HOME") or os.environ.get("USERPROFILE")

# Load credentials (Replace with your service account JSON file)
SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]
SERVICE_ACCOUNT_FILE = os.path.join(HOME_DIR, '.credentials', "oval-silicon-449118-t6-841e34d683cf.json")  # Replace with actual JSON file

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)
youtube = build("youtube", "v3", credentials=credentials)

def live_stream():

  # Insert a new livestream
  request = youtube.liveBroadcasts().insert(
      part="snippet,status,contentDetails",
      body={
          "snippet": {
              "title": "My Live Stream",
              "description": "Live streaming via Python",
              "scheduledStartTime": "2025-02-26T12:00:00Z",  # Schedule in ISO format
              "channelId": "UCPaZDuwY1sJOb5l-QHm9mDw"
          },
          "status": {"privacyStatus": "public"},
          "contentDetails": {"enableAutoStart": True},
      },
  )
  response = request.execute()

  print("Live Stream Created:", response["id"])

__all__ = {
  "live_stream": live_stream
}



authorize youtube in python?


In [ ]:

import os
import sys
import flask
import requests
from urllib.parse import urlparse, parse_qs
import json
import re

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery

HOME_DIR = os.environ.get("HOME") or os.environ.get("USERPROFILE")
# This variable specifies the name of a file that contains the OAuth 2.0
# information for this application, including its client_id and client_secret.
CLIENT_SECRETS_FILE = os.path.join(HOME_DIR, '.credentials', "youtube_secret.json")

# The OAuth 2.0 access scope allows for access to the
# authenticated user's account and requires requests to use an SSL connection.
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

app = flask.Flask(__name__)
# Note: A secret key is included in the sample so that it works.
# If you use this code in your application, replace this with a truly secret
# key. See https://flask.palletsprojects.com/quickstart/#sessions.
app.secret_key = '73954364293-ju9vifnt5233jt4fquk57hs07mh7dudg.apps.googleusercontent.com'

@app.route('/authorize')
def authorize():
  # Create flow instance to manage the OAuth 2.0 Authorization Grant Flow steps.
  flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
      CLIENT_SECRETS_FILE, scopes=SCOPES)

  # The URI created here must exactly match one of the authorized redirect URIs
  # for the OAuth 2.0 client, which you configured in the API Console. If this
  # value doesn't match an authorized URI, you will get a 'redirect_uri_mismatch'
  # error.
  flow.redirect_uri = 'https://localhost:8080' #flask.url_for('oauth2callback', _external=True)

  authorization_url, state = flow.authorization_url(
      prompt='consent',
      # Enable offline access so that you can refresh an access token without
      # re-prompting the user for permission. Recommended for web server apps.
      access_type='offline',
      # Enable incremental authorization. Recommended as a best practice.
      include_granted_scopes='true') # flow.redirect_uri

  # Store the state so the callback can verify the auth server response.
  # flask.session['state'] = state
  print("Visit this URL to authorize:", authorization_url)
  print("Paste the resulting URL:")
  line = sys.stdin.readline().strip()
  oauth2callback(line)

# https://localhost:8080/?state=O2rTe25jM5gOZ6fZBpl9A9JeLm7SYK&code=4/0AQSTgQH3rDCG1u4-CyvP1oiuKLGOMf3ZzgZRrlsaju-NKQJUPKDvB_1xalzp9_EiEXhAOw&scope=https://www.googleapis.com/auth/youtube.force-ssl
@app.route('/oauth2callback')
def oauth2callback(request_response):
  # Specify the state when creating the flow in the callback so that it can
  # verified in the authorization server response.
  # state = flask.session['state']
  parsed_uri = urlparse(request_response)
  query_params = parse_qs(parsed_uri.query)
  state = query_params['state']

  flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file(
      CLIENT_SECRETS_FILE, scopes=SCOPES)
  flow.redirect_uri = 'https://localhost:8080' # flask.url_for('oauth2callback', _external=True)

  # Use the authorization server's response to fetch the OAuth 2.0 tokens.
  authorization_response = request_response
  flow.fetch_token(authorization_response=authorization_response)

  # Store credentials in the session.
  # ACTION ITEM: In a production app, you likely want to save these
  #              credentials in a persistent database instead.
  credentials = flow.credentials
  
  credentials = credentials_to_dict(credentials)
  # flask.session['credentials'] = credentials
  print(credentials)

  # Check which scopes user granted
  #features = check_granted_scopes(credentials)
  #flask.session['features'] = features
  #return flask.redirect('/')
  save_json_with_scopes(credentials, SCOPES, os.path.join(HOME_DIR, '.credentials'))

def sanitize_filename(name):
    """Sanitize a string to be a valid filename."""
    return re.sub(r"[^\w.-]", "_", name)  # Replace invalid characters with "_"

def save_json_with_scopes(data, scopes, directory="."):
    """Save a dictionary as a JSON file, using sanitized Google OAuth scopes as filename."""
    sanitized_name = sanitize_filename("_".join(scopes))
    filename = f"{directory}/{sanitized_name}.json"

    with open(filename, "w") as f:
        json.dump(data, f, indent=4)
    
    print(f"Saved JSON to {filename}")

def credentials_to_dict(credentials):
  return {'token': credentials.token,
          'refresh_token': credentials.refresh_token,
          'token_uri': credentials.token_uri,
          'client_id': credentials.client_id,
          'client_secret': credentials.client_secret,
          'granted_scopes': credentials.granted_scopes}


__all__ = {
  "authorize": authorize,
  "oauth2callback": oauth2callback,
}



list broadcasts?


In [ ]:
import os
import re
import json
import flask
import requests

import_notebook("authorize youtube in python", globals())

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery

SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
HOME_DIR = os.environ.get("HOME") or os.environ.get("USERPROFILE")

def list_broadcasts():
  #if 'credentials' not in flask.session:
  #  return flask.redirect('authorize')
  sanitized_name = sanitize_filename("_".join(SCOPES))
  filename = f"{os.path.join(HOME_DIR, '.credentials')}/{sanitized_name}.json"
  #features = flask.session['features']
  with open(filename, "r") as f:
        credentials_dict = json.load(f)

  if(not credentials_dict or not os.path.exists(filename)):
    authorize()

  # Load credentials from the session.
  credentials = google.oauth2.credentials.Credentials(
    credentials_dict["token"],
    refresh_token = credentials_dict["refresh_token"],
    token_uri = credentials_dict["token_uri"],
    client_id = credentials_dict["client_id"],
    client_secret = credentials_dict["client_secret"],
    scopes = SCOPES)

  youtube = googleapiclient.discovery.build(
      API_SERVICE_NAME, API_VERSION, credentials=credentials)

  broadcasts = youtube.liveBroadcasts().list(part='id,snippet', mine=True).execute()

  return broadcasts
  # Save credentials back to session in case access token was refreshed.
  # ACTION ITEM: In a production app, you likely want to save these
  #              credentials in a persistent database instead.
  #flask.session['credentials'] = credentials_to_dict(credentials)

  #return flask.jsonify(**broadcasts)


def sanitize_filename(name):
    """Sanitize a string to be a valid filename."""
    return re.sub(r"[^\w.-]", "_", name)  # Replace invalid characters with "_"


__all__ = {
  "list_broadcasts": list_broadcasts
}



create live stream?


In [ ]:
import os
import re
import json
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
HOME_DIR = os.environ.get("HOME") or os.environ.get("USERPROFILE")

def create_livestream(youtube):

    # Load OAuth 2.0 credentials from a saved token file
    sanitized_name = sanitize_filename("_".join(SCOPES))
    filename = f"{os.path.join(HOME_DIR, '.credentials')}/{sanitized_name}.json"
    #credentials = Credentials.from_authorized_user_file(filename, SCOPES)


    with open(filename, "r") as f:
        credentials_dict = json.load(f)

    # Load credentials from the session.
    credentials = Credentials(
    credentials_dict["token"],
    refresh_token = credentials_dict["refresh_token"],
    token_uri = credentials_dict["token_uri"],
    client_id = credentials_dict["client_id"],
    client_secret = credentials_dict["client_secret"],
    scopes = SCOPES)
    # Build the YouTube API client
    youtube = build("youtube", "v3", credentials=credentials)

    print(youtube)
    request = youtube.liveStreams().insert(
        part="snippet,cdn,status",
        body={
            "snippet": {
                "title": "My Live Stream",
                "description": "Streaming via API"
            },
            "cdn": {
                "frameRate": "30fps",
                "resolution": "1080p",
                "ingestionType": "rtmp"
            },
            "status": {
                "streamStatus": "active"
            }
        }
    )

    response = request.execute()
    return response['cdn']['ingestionInfo']['streamName']

def sanitize_filename(name):
    """Sanitize a string to be a valid filename."""
    return re.sub(r"[^\w.-]", "_", name)  # Replace invalid characters with "_"


__all__ = {
  "create_livestream": create_livestream
}


list live stream?


In [ ]:
import os
import re
import json
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
HOME_DIR = os.environ.get("HOME") or os.environ.get("USERPROFILE")

def list_livestream(youtube):
    
    # Load OAuth 2.0 credentials from a saved token file
    sanitized_name = sanitize_filename("_".join(SCOPES))
    filename = f"{os.path.join(HOME_DIR, '.credentials')}/{sanitized_name}.json"
    #credentials = Credentials.from_authorized_user_file(filename, SCOPES)


    with open(filename, "r") as f:
        credentials_dict = json.load(f)

    # Load credentials from the session.
    credentials = Credentials(
    credentials_dict["token"],
    refresh_token = credentials_dict["refresh_token"],
    token_uri = credentials_dict["token_uri"],
    client_id = credentials_dict["client_id"],
    client_secret = credentials_dict["client_secret"],
    scopes = SCOPES)
    # Build the YouTube API client
    youtube = build("youtube", "v3", credentials=credentials)

    request = youtube.liveStreams().list(
        part="snippet,cdn,status",
        mine=True)

    response = request.execute()
    print(response)
    filtered_streams = [
        stream for stream in response["items"]
        if stream["status"]["streamStatus"] != "active"
    ]

    return filtered_streams[0]['cdn']['ingestionInfo']['streamName']

def sanitize_filename(name):
    """Sanitize a string to be a valid filename."""
    return re.sub(r"[^\w.-]", "_", name)  # Replace invalid characters with "_"


__all__ = {
  "list_livestream": list_livestream
}